In [56]:
import subprocess
from pyspark.sql.types import IntegerType,FloatType
from functools import reduce
from pyspark.sql import DataFrame,SparkSession
from pyspark.sql.functions import sum
from pyspark.sql.functions import col



In [57]:
def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True,inferSchema = True)
    # rdd=rdd.drop("axlespacings", "axleweights", "numberofaxles","validitycode","duration","temperature","weight","hour","minute","second","millisecond")
    return rdd

In [58]:
def get_Dataframe(x):
    p=subprocess.Popen(x,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT)
    result=list()
    for line in p.stdout.readlines():
        file = line.decode().strip()

        df= hdfs_file_read(file)


        df=df.filter((df.classname  == "HGV_RIG") | (df.classname  == "HGV_ART")).groupBy('cosit').count()
        df=df.sort('count',ascending=False)

        result.append(df)

    union_df = reduce(DataFrame.unionAll, result)
    union_df=union_df.groupBy('cosit').sum('count').sort('sum(count)',ascending=False).limit(10)
#     print(result)
    return union_df

In [59]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/processedVehicleData| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownProcessedVehicleData| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])
        
week0.show()
week1.show()

+------+----------+
| cosit|sum(count)|
+------+----------+
|  1502|     87976|
|  1508|     85061|
|  1501|     81404|
|  1503|     79830|
|  1500|     76481|
|200723|     69288|
|  1073|     61825|
|  1012|     59196|
|  1071|     59036|
|  1072|     58999|
+------+----------+

+------+----------+
| cosit|sum(count)|
+------+----------+
|   997|    118792|
|  1508|     58018|
|  1502|     55658|
|  1501|     52465|
|  1503|     50153|
|  1070|     40021|
|200723|     39655|
|  1012|     39046|
|  1500|     38597|
|  1071|     36490|
+------+----------+

